In [18]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeConfigurationError: You haven't specified an Api-Key.

: 

In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Smith explains concepts clearly and is always willing to help students during office hours.'},
 {'professor': 'Prof. Emily Brown',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Prof. Brown is amazing! Her classes are engaging, and she makes complex topics easy to understand.'},
 {'professor': 'Dr. Robert Davis',
  'subject': 'Mathematics',
  'stars': 3,
  'review': 'Dr. Davis is knowledgeable, but his lectures can be a bit dry at times.'},
 {'professor': 'Dr. Alice Green',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Dr. Green is very passionate about her subject and makes learning fun.'},
 {'professor': 'Prof. Michael Johnson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Prof. Johnson is an excellent teacher who really cares about his students' success."},
 {'professor': 'Dr. Sarah Wilson',
  'subject': 'History',
  'stars': 2,
  'review': 'Dr. Wilson knows her material

: 

In [ ]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metedata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

: 

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

NameError: name 'pc' is not defined

: 

In [ ]:
index.describe_index_stats()

NameError: name 'index' is not defined

: 